# What's new in the Forecastwrapper V2

• No more seperate objects `Weather_Days` and `Weather_Hours`, just `Weather`

• The `Weather` object has methods `days()` and `hours()` which return a dataframe in the desired resolution. (We chose this approach because now you only need 1 call to forecast.io to get both resolutions, in V1 you needed 2 calls if you wanted both)

• More verbose and pythonicly named arguments. Eg. the flag `heatingDegreeDays` is now called `include_heating_degree_days`

• The temperature equivalent is now by default included in the daily dataframe. It can be excluded by using the flag `include_temperature_equivalent = False`

# Demo of the forecast.io wrapper to get past and future weather data

Important: you need to register for an apikey here: https://developer.forecast.io/  Put the key you obtain in the opengrid.cfg file as follows:

    [Forecast.io]
    apikey: your_key

In [ ]:
import os, sys, inspect
import pandas as pd
import charts

In [ ]:
# add the sccript path to opengrid to sys.path
script_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.join(script_dir, os.pardir, os.pardir))

from opengrid.library.config import Config

## Get Forecast.io API key from config file

In [ ]:
config = Config()

#get Forecast.io API Key
api_key = config.get('Forecast.io', 'apikey')

## Import API wrapper module

In [ ]:
from opengrid.library import forecastwrapperV2

# Get weather data in daily and hourly resolution

To get started, create a Weather object for a certain location and a period

In [ ]:
start = pd.Timestamp('20150813')
end = pd.Timestamp('20150816')

In [ ]:
Weather_Ukkel = forecastwrapperV2.Weather(api_key=api_key, location='Ukkel', start=start, end=end)

You can use the methods `days()` and `hours()` to get a dataframe in daily or hourly resolution

In [ ]:
Weather_Ukkel.days()

In [ ]:
Weather_Ukkel.hours()

### Degree Days

Daily resolution has the option of adding degree days.
By default, the temperature equivalent and heating degree days with a base temperature of 16.5°C are added.

Heating degree days are calculated as follows:

$$heatingDegreeDays = max(0 , baseTemp - 0.6 * T_{today} + 0.3 * T_{today-1} + 0.1 * T_{today-2} )$$

Cooling degree days are calculated in an analog way:

$$coolingDegreeDays = max(0, 0.6 * T_{today} + 0.3 * T_{today-1} + 0.1 * T_{today-2} - baseTemp )$$

Add degree days by setting the flag `include_heating_degree_days` or `include_cooling_degree_days` and supplying `heating_base_temperatures` and/or `cooling_base_temperatures` as a list (you can add multiple base temperatures, or just a list of 1 element)

#### Get some more degree days

In [ ]:
Weather_Ukkel.days(include_heating_degree_days = True,
                   heating_base_temperatures = [15,18],
                   include_cooling_degree_days = True,
                   cooling_base_temperatures = [18,24]).filter(like='DegreeDays')

### Daytime Averages

Daily resolution has the option of adding average values during the daytime only (between sunrise and sunset). Implemented are daytimeCloudCover and daytimeTemperature, and are included by default. They can be excluded by setting `include_daytime_cloud_cover = False` or `include_daytime_temperature = False`.

By default, daily weather data is extended with *daytimeCloudCover* and *daytimeTemperature*.  These are the averaged cloud cover and temperature **between sunrise and sunset**. 

In [ ]:
Weather_Ukkel.days().filter(like='daytime')

# Hourly resolution example

Location can also be coördinates

In [ ]:
start = pd.Timestamp('20150916')
end = pd.Timestamp('20150918')
Weather_Brussel = forecastwrapperV2.Weather(api_key=api_key, location=[50.8503396, 4.3517103], start=start, end=end)
Weather_Boutersem = forecastwrapperV2.Weather(api_key=api_key, location='Kapelstraat 1, 3370 Boutersem', start=start, end=end)

In [ ]:
df_combined = pd.merge(Weather_Brussel.hours(), Weather_Boutersem.hours(), suffixes=('_Brussel', '_Boutersem'), 
                       left_index=True, right_index=True)

In [ ]:
charts.plot(df_combined.filter(like='cloud'), stock=True, show='inline')

# Deal with different timezones

By default, all dataframes are localized to the timezone of the specified location.
If you want, you can specify the tz argument to get all dataframes localized to a specified timezone.

In [ ]:
start = pd.Timestamp('20150815')
end = pd.Timestamp('20150816')

In [ ]:
Weather_London = forecastwrapperV2.Weather(api_key=api_key, location='London', start=start, end=end, tz='Asia/Singapore')
Weather_Brussels = forecastwrapperV2.Weather(api_key=api_key, location='Brussels', start=start, end=end, tz='Asia/Singapore')

In [ ]:
Weather_London.days()

In [ ]:
Weather_Brussels.days()